# Fitting Models to Data: Frequentist vs. Bayesian Approach

<img src="http://homepages.wmich.edu/~wlacefie/images/evobayes.jpg">

## Step 1: Download and install the needed packages 

You'll need 'emcee' and 'corner' for the tutorial below. Your computer may already have these programs installed. You can check if it does by seeing if the cell below runs without error. The lines of interest are:
* import emcee 
* import corner

If the cell below gives you an error, then:
* Click <a href="http://dan.iel.fm/emcee/current/user/install/">here</a> for information on emcee. There are install instructions on that page.  I recommend to use the command line 'pip' to install. 
* Click <a href="https://github.com/dfm/corner.py">here</a> for information on corner.  Again, I recommend using pip to install corner.

In [ ]:
#After you've downloaded the above packages, import the needed libraries
#you'll need numpy, matplotlib, pyplot, emcee, corner (if necessary, see previous workshops for how to import these)
---

#we also want these
from scipy.optimize import leastsq
import multiprocessing as multi
nthreads = multi.cpu_count()

## Step 2: Exploring Random Number Generation

First let's talk about drawing values randomly in general.

Computers don't know how to draw truly random numbers. Instead they generate some large sequence of numbers that appear random, but are really deterministic. Usually you have to define a "seed" value to define where to start the sequence.

In python you don't actually have to define the "seed" value explicitly, but you can in order to keep your "random" draw the same each time you run the code.  

In [ ]:
#Defining the "seed" for numpy's random function
numpy.random.seed(seed = 1234567)

Now, let's draw randomly from a Gaussian (aka 'Normal') distribution. <br>
Note: A Gaussian distribution looks like: <img src="http://zoonek2.free.fr/UNIX/48_R/g587.png" width=300>

In [ ]:
#Numpy has a built-in function to draw random numbers from a Gaussian
#define the Gaussian using the mean, standard deviation (stdev), 
#  and choose some number of values (Nvals) that you want to draw
mean =  ---
stdev = ---
Nvals = ---
rg = numpy.random.normal(mean, stdev, size = Nvals)

#print the mean, min and max of this list of random mumbers
print 'mean,min,max of the random sample', ---, ---, ---

In [ ]:
#plot a histogram of these values
n, bins, patches = pyplot.hist(rg, 50, color = 'b', alpha = 0.6, range = [mean - 5.*stdev, mean + 5.*stdev])

#overplot a dashed line that shows the mean value
pyplot.plot(---)

pyplot.xlabel("value")
pyplot.ylabel("N")
pyplot.draw()

In [ ]:
#How many values are within 1,2,3 sigma of the mean for this numpy.random.normal function?
#Hint: you can use numpy.where to find all the values that are within some range of the mean
for fac in [1,2,3]:
    ---
    print "percent inside of %f times sigma = %f" % (fac, ---)

#### Question: Do the numbers for sigma match up to the <a href="http://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule">right answer for a gaussian distribution</a>?

If not exactly, why not? What could you change to get closer to the right answer? Make this change above, and see if you can come closer to the correct values.

In [ ]:
#What if you didn't have this easy numpy.random.normal function?
#You could instead sample a Gaussian using this "Box-Muller" procedure 
#with two random numbers between 0 and 1. 
#In python you can draw numbers from a uniform distribution with numpy.random.random()
#This method is very useful, e.g., in C or fortran (and many othe codes)
rg_bm = []
for i in range(Nvals):
#draw your first random number
    x1 = ---
#draw your second random number    
    x2 = ---
#just in case
    while (x1 == 0): x1 = numpy.random.random()
    while (x2 == 0): x2 = numpy.random.random()
#this draws from a Guassian distribution
    rg_bm.append(stdev * (-2.*numpy.log(x1))**0.5 * numpy.cos(2.*numpy.pi*x2) + mean)
    
rg_bm = numpy.array(rg_bm) #just to keep them in the same format

In [ ]:
#How many values are within 1,2,3 sigma for the "Box-Muller" procedure?
for fac in [1,2,3]:
    ---
    print "percent inside of %f times sigma = %f" % (fac, ---)

In [ ]:
#plot a histogram of these values

#first the numpy.random.normal results
---
#now the Box-Muller results
---

#overplot a dashed line that shows the mean value
---

#overplot dotted lines that show the 1,2,3 sigma values
---
    
pyplot.xlabel("value")
pyplot.ylabel("N")
pyplot.draw()

## Step 3: Generating Observed Orbit Data for a Spectroscopic Binary

Now that we know a little more about random number generation, let's apply it to an astrophysical context. 

Let's simulate observed data for a spectroscopic binary with a time series of radial velocites. We want to fit an orbital solution to these observations.

The image illustrates binary stars' motion over time, their spectra, and radial velocity. <img src="http://plato.acadiau.ca/courses/phys/1523/jan18/specbiny.gif">

### Generate 'observed' heliocentric radial velocity of a spectroscopic binary. 

In [ ]:
#The following code is based on helio_rv.pro, from IDL's astronomy library 
#note: omega is given in degrees

#JD = Julian Date
#for your birthday in JD, check out http://aa.usno.navy.mil/data/docs/JulianDate.php

def helio_rv(JD, params, lim = 1.e-8, nlim = 1000.):
    T, P, V0, K, e, omega = params
    
    #safety check
    rv = [1e10 for i in JD]
    if (e >= 0. and e < 1. and P > 0. and K > 0. and omega >= 0. and omega <= 360.):

        # Calculate the approximate eccentric anomaly, E1, via the mean anomaly, M.
        # (from Heintz DW, "Double stars", Reidel, 1978)
        M = numpy.array([2. * numpy.pi * ( (x - T) / P % 1.) for x in JD])
        E1 = numpy.array([x + e*numpy.sin(x)  + ( (e**2.) * numpy.sin(2.*x) / 2. ) for x in M])

        # Refine this estimate using formulae given by Reidel.
        test = 2*lim
        i = 0
        nlim = 1000.
        while (test > lim):
            i += 1
            E0 = E1
            M0 = E0 - e * numpy.sin(E0)
            E1 = E0 + (M - M0) / (1. - e*numpy.cos(E0))
            use = numpy.where(E1 != 0)[0]
            if (len(use) > 0):
                test = max(abs( (E1[use] - E0[use])/E1[use]) )
            else:
                test = lim
            if (i > nlim):
                test = lim
                    
        # Calculate nu
        nu = 2. * numpy.arctan( ((1. + e)/(1. - e))**0.5 * numpy.tan(E1/2.) )

        # Calculate radial velocities
        rv = (K* (numpy.cos(nu + omega*numpy.pi/180.) + (e*numpy.cos(omega*numpy.pi/180.)) ) ) + V0
    
    return rv

In [ ]:
#Set the number of observations (feel free to change this)
Nobs = 20

#Set the observation dates
drange = 50.
JD = [drange*x  for x in numpy.random.random(Nobs)]

In [ ]:
#Here we set the true orbital parameters and determine the true radial velocities for the primary star.
#Note: below you'll add 'noise' to account for measurement precision

#Parameters (feel free to change these)
#For simplicity in this example, we will only consider (near) circular orbits 
#  (I've found that having exactly zero for e and omega can "break" the fitters for some reason)
#  but feel free to explore non-zero eccentricites on your own after completing this workshop!
T0 = drange/5.  #days (generally in JD) at periastron
P = 0.7*drange   #period in days
e = 0.001   #eccentricity
V0 = 25.    #center-of-mass velocity km/s
K = 10.     #amplitude of signal km/s (NOTE: this encodes a lot of hidden information, e.g. masses, inclination)
omega = 0.001 #longitude at peri (this is not well defined for e = 0, because there is no peri)
params = [T0, P, V0, K, e, omega]
params_name = ['T0', 'P', r'$\gamma$', 'K', 'e', r'$\omega$']

#Obtaining the true radial velocities based on the parameters above
RV_true = helio_rv(JD, params) #km/s

In [ ]:
#Now add in noise to account for the measurement precision  (i.e., magnitude of the typical error)
noise_lvl = 0.5 #km/s, single measurement precision, (i.e. the error on each measurement)

#in order to add this noise to the true observations, you need to draw from a Gaussian with sigma = noise_lvl
RV_obs = RV_true + ---

In [ ]:
#calculate the residuals from the fit:
#create a list that contains the difference between the true model (RV_true) and the observed data (RV_obs)
OC = ---

In [ ]:
#Plot the true curve (solid line) versus the generated observations (symbols) 
#The third plot show the residuals (i.e., the difference between true curve and observations) versus the phase

#to plot the curves
Npts = 5000
JD_x = [x/float(Npts)*(max(JD) - min(JD)) + min(JD) for x in range(Npts)]

f1 = pyplot.figure(figsize=(7,7))

#Plot #1
#RV vs. JD
ax1 = pyplot.subplot(211)
#first, plot the curve using the helio_rv function
ax1.plot(JD_x, helio_rv(JD_x, params),'g') #true curve
#now plot the observed RVs
ax1.errorbar(---, ---, yerr = ---, fmt='.',c='g') #observations
#label the plot
ax1.set_xlabel('JD')
ax1.set_ylabel('RV')
ax1.set_ylim([min(RV_obs),max(RV_obs)])

#Plot #2
#O-C vs. JD
ax2 = pyplot.subplot(212)
#plot the O-C residuals for the observed RVs and the helio_rv result
ax2.errorbar(---, ---, yerr = ---, fmt='.',c='g')
#plot a dashed line through O-C = 0 for reference
---
#label the plot
ax2.set_xlabel('JD')
ax2.set_ylabel('(O-C)')
ax2.set_ylim([-5.*noise_lvl,5.*noise_lvl])

pyplot.draw()

## Step 4: Determine the Best Fit using the 'Frequentist' Approach

### Part A: "Frequentists" generally <a href="https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic">minimize the chi^2</a> (or sometimes the residuals) to fit a function to data. Let's do that!

In [ ]:
#Define the chi^2 and reduced chi^2, which are useful indicators of your goodness-of-fit
def chi2(ps, args):
#we will send this function four values
#in our case:
#  x contains the JD values 
#  obs contains the RV observations 
#  sigma contains the uncertainties on the Rv observations
#  func contains the function that calculates the RV curve (helio_RV)
#But note that this is a general function that you could use in any case that you calculate chi^2
    x,obs,sigma,func = args
#write this function to return the result of the equation on that wikipidea page 
#  linked above shown in the section "Calculating the test-statistic"
    ---
    return ---

def chi2_red(ps, args):
    x,obs,sigma,func = args
#now we want the reduced chi^2, which is chi2/dof, 
# dof is the number of degrees of freedom in your fit = number of observations - number of parameters
    ---
    return ---

In [ ]:
#compare the reduced chi^2 to our observations (with Gaussian errors).  
#This should be about 1. 
args = (JD, RV_obs, noise_lvl, helio_rv)
#I'm just defining some nice looking output line that we can use later on, and you'll fill in values below
#-- you'll want to send this print statement tuples
print '%-10s [%10s %10s %10s %10s %10s %10s] %10s' % (("input",)+tuple(params_name)+("chi^2",))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("true",)+tuple(params)+(chi2_red(params,args),))

### In order to fit to these data by finding the minimum chi^2 value, we will need an initial guess at the parameters...<br>

recall params [T0,P,VO,K,e,omega]:
* T0 = days (generally in JD) at periastron
* P = period in days
* V0 = center-of-mass velocity km/s
* K = amplitude of signal km/s 
* e = eccentricity
* omega = longitude at peri

In [ ]:
#let's make a semi-informed, initial guess
guess0 = [(max(JD)-min(JD))/2., 
          (max(JD)-min(JD))/2.,
          numpy.mean(RV_obs),
          (max(RV_obs)-min(RV_obs))/2.,
          0.0,
          0.0]

#check the reduced chi^2 to our guess
print '%-10s [%10s %10s %10s %10s %10s %10s] %10s' % (("input",)+tuple(params_name)+("chi^2",))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("true",)+tuple(params)+(chi2_red(params,args),))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("guess",)+tuple(---)+(---,))

See how the reduced chi^2 for our guess is >> 1? This is bad!

### To do: Input different parameter values in guess0 in the cell above (by changing the numbers) and rerun the cell to see how the chi^2 changes.

Question:<br> 
Are you able to get a better chi^2 than we got above with our initial guess?

## In the steps below we'll try to automate minimizing the chi^2 starting from our initial-guess parameters.

We will use scipy's <a href="http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.leastsq.html"> leastsq</a> function.

In [ ]:
#let's try to minimize the chi^2 starting from our initial guess

#we will use leastsq from scipy for this, which performs a least squares fit (e.g., minimizing the chi^2)

#leastsq requires chi^2 as a list so I'll define it here 
def chi2list(ps, args):
    x,obs,sigma,func = args
    OC = (obs - func(x,ps))
    return [o**2./sigma**2. for o in OC]
 
#here's how you can run leastsq. See the link above for more information on leastsq. 
fres = leastsq(chi2list, guess0, args = (args,), full_output = 1)

# pleastsq0 = Initial parameter estimation based on trying to minimize chi^2
pleastsq0 = fres[0]
print '%-10s [%10s %10s %10s %10s %10s %10s] %10s' % (("input",)+tuple(params_name)+("chi^2",))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("true",)+tuple(params)+(chi2_red(params,args),))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("guess",)+tuple(---)+(---,))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("fit0",)+tuple(---)+(---,))

#### The chi^2 for the fit is better than our guess, but may still pretty bad, depending on how good your guess was (remember, we want a reduced chi^2 near 1). 

Below we plot the result so you can see by eye that the fit still isn't very good. 

In [ ]:
#Add this minimized chi^2 fit to the figures (fit shown in cyan)

f1 = pyplot.figure(figsize=(7,7))

#Plot #1
#RV vs. JD
ax1 = pyplot.subplot(211)
---

#Overplot the minimized chi^2 fit
ax1.plot(JD_x, helio_rv(---, ---), 'c') 

#Plot #2
#O-C vs. JD
ax2 = pyplot.subplot(212)
---

#Overplot the minimized chi^2 fit
# Subtract the true radial velocities from the leastsq fit radial velocities
OC_fit1 = ---
ax2.errorbar(---, OC_fit1, yerr = noise_lvl, fmt='.',c='c')

pyplot.draw()

### Possibly still pretty bad, (at least it was for me!)

### Part B: Try a Range of Initial Guesses to Try and Get a Better Fit

Rather than just picking one guess, we can have python select any number of random initial guesses for us and run them through the leastsq fitter to see if any have a reduced chi^2 ~ 1.

Here 'walkers' = initial guesses.  (We'll explain this terminology.)

In [ ]:
# Set the number of walkers to use 
nwalkers = 20
#note: emcee (below) requires an even number of walkers (initial guesses), 
#and the number of walkers must be at least twice the number of parameters that we're fitting (which is 6)

#To help guide the fit, let's add some bounds for the parameters 
#NOTE: the leastsq function in scipy does not allow bounds on parameters (though other fitters do)
#      we will simply use these bounds to define our walkers
bnds0 = ((min(JD), max(JD)), 
        (1., max(JD)-min(JD)), 
        (min(RV_obs), max(RV_obs)), 
        (noise_lvl, 5.*(max(RV_obs)-min(RV_obs))), 
        (0., 0.01), 
        (0., 0.01))


# we want a list of lists here such that we have initPos = [ [params1], [params2], ... ]
# and params# contains all the parameters from our RV curve = T0, P, gamma, K, e, omega'
# each value in params# should be drawn from a random uniform distribution between the relevant bnds0 defined above
#
# Hint: I found it easier to first select the parameters in an array where a given "row" contains 
# nwalkers random draws for the given param value, and to then transpose this matrix so that I can get 
# the list of guesses that I said we need. 

initPos = ---


In [ ]:
#Now let's step through these guesses and do the chi^2 minimization on each
#Note: some don't finish successfully (see last column)

#lets define some dictionary to hold the results from these fits
fits = {"chi2":[], "params":[], "pcov":[]}

#just for reference
print '%-10s [%10s %10s %10s %10s %10s %10s] %10s' % (("input",)+tuple(params_name)+("chi^2",))
print '%-10s [%10f %10f %10f %10f %10f %10f] %10f\n' % (("true",)+tuple(params)+(chi2_red(params,args),))

for p in initPos:
#use leastsq to get the best fit given this guess, p
    fres = ---
    
    print '%-10s [%10f %10f %10f %10f %10f %10f] %10f' % (("guess"+str(i+1),)+tuple(p)+(chi2_red(p,args),))
    print '%-10s [%10f %10f %10f %10f %10f %10f] %10f\n' % (("fit"+str(i+1),)+tuple(---)+(---,))

    fits["chi2"].append(chi2_red(fres[0],args))
    fits["params"].append(fres[0])
    fits["pcov"].append(fres[1])

### To do: Look at the output above showing the parameter values and the chi^2 values. Can you find the smallest chi^2 value in the list? 

In the step below, you'll automate finding the smallest chi^2 value from this outputted list. 

In [ ]:
#Identify the best-fit from this range of guesses

# numpy.nanargmin finds the minimum value in an array (which isn't 'nan' or 'inf')
pbest = numpy.nanargmin(fits["chi2"])

# pleastsq gives the parameters corresponding to the minimum chi^2 value (i.e., pbest)
pleastsq = ---
#print that out
print '%-24s [%10f %10f %10f %10f %10f %10f] %10f' % (("second fit attempt",)+tuple(---)+(---,))


### If you don't get a result with chi^2 ~ 1, then rerun the previous 3 cells until a random guess gets close enough to give you a good fit.

Below we plot the result so you can see by eye that the fit still isn't very good. 

In [ ]:
#Plot this best-fit

#Purple: best-fit based on larger sample of random guesses
#Cyan: first attempt at the least-squares fit
#Green = true curve

f1 = pyplot.figure(figsize=(7,7))

#Plot #1
#RV vs. JD
ax1 = pyplot.subplot(211)
---

#least-squares fit from larger sample of random guesses
---

#first least-squares fit
---

#Plot #2
#O-C vs. JD
ax2 = pyplot.subplot(212)
---

#first least-squares fit
---

#least-squares fit from larger sample of random guesses
OC_fit2 = ---
ax2.errorbar(---)

pyplot.draw()

### Now the chi^2 should be much closer to 1, and the plots should show the fit is quite good as well.

### Now, what are the errors on these fit parameters?

Note: occasionally I've found that leastsq does not return a covariance matrix, and in that case the code below will fail.  If that happens, rerun the above cells to find a new solution that has a covariance matrix.

In [ ]:
#get the errors from the covariance matrix, 
#taken from http://stackoverflow.com/questions/14581358/getting-standard-errors-on-fitted-parameters-using-the-optimize-leastsq-method-i
pfit = fits["params"][pbest]
pcov = fits["pcov"][pbest]
pcov = pcov * chi2_red(pfit, args)
error = [] 
for i in range(len(params)):
    try:
        error.append( numpy.absolute(pcov[i][i])**0.5)
    except:
        error.append( 0.00 )

for i,p in enumerate(pfit):
    name = params_name[i]
    print '%-10s : %10f  +/- %10f ' % ((params_name[i],)+(p, error[i],))



### Questions:
* How well did this match your input values?     

* In particular, did you get the right T0?  If not, why, and is this a problem?   

* Did you get the right omega?  Why might this be hard to fit for the system we chose?

### Note: often the errors derived in this manner are either underestimated or otherwise unrealistic because the uncertainties on the observed data are poorly known (for instance see the discussion <a href="http://dan.iel.fm/emcee/current/user/line/"> here</a>).

## Step 5: Fit a Model to the Data using the Bayesian Method

Another way to fit a model to data is through the <a href="http://en.wikipedia.org/wiki/Bayes%27_theorem">Bayesian</a> method. Instead of finding a "best-fit" solution, "Bayesians" want to find the distribution of solutions, called the "posterior". 

Bayes' theorem is stated mathematically as the following equation:

$P(A|B) = \frac{P(B | A)\, P(A)}{P(B)}$,
where A and B are events.

* $P(A)$ and $P(B)$ are the probabilities of A and B without regard to each other.
* $P(A | B)$, a conditional probability, is the probability of A given that B is true.
* $P(B | A)$, is the probability of B given that A is true.

In practice, $B$ is the model (here, the helio_rv function), and $A$ is the data (the RV observations).  We want to know the $P(A|B)$, or, in other words, how well the model describes the data.  We generally don't know $P(B)$, the instrinsic probability that our model is "correct".  We call $P(A)$ the "prior", and this defines our understanding of the instrinsic probability that a given parameter in our model has a certain value (for instance, maybe this parameter, say the orbital period, is believed to be drawn from a Guassian distribution).  $P(B|A)$ is the "likelihood", and can be thought of as a goodness of fit value for the model given the data.

Python has a very useful tool, called 'emcee' that will run a <a href="http://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo">Markov Chain Monte Carlo (MCMC)</a> sampling, and return the posterior to us. Let's set this up!

For additional information on 'emcee' check out <a href="http://dan.iel.fm/emcee/current/user/line/">this tutorial</a>. 

In [ ]:
#For Guassian noise you can write the likelihood as exp(-chi^2/2)
#emcee uses ln(likelihood) so...
def lnlike(ps, args):
    return -0.5*chi2(ps,args)

Note: We can set bounds on the parameter values (like above) to create flat priors, if we set a prior value of 1 within the range and 0 outside of it.

In [ ]:
#To help guide the fit, let's add some bounds for the parameters 
bnds = ((min(JD), max(JD)), 
        (1., max(JD)-min(JD)), 
        (min(RV_obs), max(RV_obs)), 
        (noise_lvl, 5.*(max(RV_obs)-min(RV_obs))), 
        (0., 0.999), 
        (0., 360.))

def lnprior(ps):
    lp = 0. #ln(1)
    for i,pp in enumerate(ps):
        if (pp < bnds[i][0] or pp > bnds[i][1]):
            return -numpy.inf #ln(0) if outside the bounds

    return lp

In [ ]:
#now construct the probability as priors*likelihood (but again in the log)
def lnprob(ps, args):
    lp = lnprior(ps)
    if not numpy.isfinite(lp):
        return -numpy.inf
    return lp + lnlike(ps, args)

### <a href="http://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo">Markov Chain Monte Carlo (MCMC)</a> is a method for efficiently searching through a large parameter space (like we have here).  

It starts a number of "walkers" at their initial guesses, then walks them towards the best fit value(s) based on "jump" criteria (the art in the MCMC method) that allow the walker to move in this direction.   Eventually the walkers should reach this best fit "peak" in the posterior distribution, and spend most of their time mapping out this region.  When we get the "sampler" back, we will see where the walkers have been.  The "chain" is the path that a given walker takes through the posterior distribution.  With enough walkers and enough samples, the walkers will map out the posterior distribution for us. 

In [ ]:
#initialize the mcmc sampler
ndim = len(params)
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args = (args,), threads = nthreads)

In [ ]:
#draw initial walkers from the covariance matrix given by the least squares fit
#we don't have to do it this way (for instance we could simply take the random walkers we had before),
#but this will produce a good fit more easily -- and we've already done the work above, so let's use that result.
def getwalkers(best, cov, walkers, nwalkers, n=10):
    pInits = numpy.random.multivariate_normal(best, cov*n, nwalkers)
    for j,p in enumerate(pInits):
        for i,a in enumerate(p):
            if (a > bnds[i][1] or a < bnds[i][0]):
                pInits[j][i] = walkers[j][i]
    return pInits

walkers = getwalkers(pfit, pcov, initPos, nwalkers)
f4 = corner.corner(walkers, labels = params_name, truths = params)


In [ ]:
#Run our MCMC :
#(this will take some time if you have a large N. I'd recommend starting with a smaller N first to test things.)
sampler.reset()

#number of iterations to take
N = 1000

#Only record every Nthin sample
Nthin = 10


#Run the MCMC (using the initial random guesses, or drawn from the leastsq covariance matrix)
#sampler.run_mcmc(initPos, N, thin = Nthin)
sampler.run_mcmc(walkers, N, thin = Nthin)

In [ ]:
# We now have access to our posterior sample through the sampler object:  
print "The shape of the sampler.chain is (Nwalker , Nsamples, Nparams): ", sampler.chain.shape
print "The shape of the chain.flatchain is (Nwalker x Nsamples, Nparams): ", sampler.flatchain.shape

### The "burn in"

In general, within MCMC, the walkers need a little bit of time to determine their direction of travel, i.e., to start climbing up toward the best-fit value(s).  We call this initial bit the "burn-in", and we generally discard that because it is not informative about the true posterior distribution.

In [ ]:
#consider the first nburn as a "burn-in" and create the posterior distribution
nburn = 20
samples = sampler.chain[:, nburn:, :].reshape((-1, ndim))

### Plot the chains

In [ ]:
pyplot.figure(figsize=(10,20))

subn = len(params)*100+11

#each color shows a different walker (though some colors are probably repeated)
#solid black horizontal lines show the true values
#to the left of the dashed vertical black lines is the (discarded) burn in
for walker in range(sampler.chain.shape[0]):
    for i,p in enumerate(params):
        ax = pyplot.subplot(subn+i)
        ax.plot(sampler.chain[walker,:,i])
        ax.plot([0,len(sampler.chain[walker,:,i])],[p,p],'k', linewidth=2)
        ax.plot([nburn,nburn],[min(sampler.chain[walker,:,i]),max(sampler.chain[walker,:,i])],'k--', linewidth=2)
        ax.set_ylabel(params_name[i])

pyplot.draw()


### Plot the posterior distribution resulting from these chains

In [ ]:
# We can also plot our posterior distributions on a sweet corner plot.
f4 = corner.corner(---)


### Plot the distribution of chi^2 values from the fit as a check

In [ ]:
#Determine the distribution of reduced chi^2 values for the MCMC run
chi2a = [--- for --- in samples]

#Plot the chi^2 distribution of MCMC fits
pyplot.figure(figsize=(7,5))

#plot a histogram of the reduced chi^2 values
---

pyplot.xlabel(r'log($\chi^2$)')
pyplot.ylabel('N')

pyplot.draw()

### Plot the emcee fit on top of our other fits

In [ ]:
#Plot this best-fit(s)

#Purple: best-fit based on larger sample of random guesses
#Cyan: first attempt at the least-squares fit
#Green:  true curve
#Orange: MCMC fit

---

#MCMC fit
#for this one we want to select some number of random samples from the posterior to plot
Nvals = ---
for i in range(Nvals):
#choose some random list position
    pos = ---
    pMCMC = samples[pos]

    ax1.plot(---, 'orange', alpha = 0.2) 
    OC_MCMC = ---
    ax2.errorbar(---, c='orange', alpha = 0.2)

pyplot.draw()

### A benefit of MCMC is that you can be clear about your uncertainties.

In [ ]:
#What do we quote as our results when we publish and share with our colleagues?

#We could quote the following. This will give the 50% percentile and 
#then the +/- "1 sigma" error bars
pr_lo = 16.
pr_mi = 50.
pr_hi = 84.

pout = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*numpy.percentile(samples, [pr_lo, pr_mi, pr_hi], axis=0)))
for i,p in enumerate(pout):
    print '%-10s : %10f  +%10f  -%10f' % ((params_name[i],)+tuple(p))  

### Questions: 
* How do these uncertainties compare to those that you found for the leastsq fit?   
    
* Why do we have different + and - errors here, and only one +/- value above?

* Which uncertainties do you trust more?  Why?

#### Note: that you have more information in the Bayesian fit than just the uncertainties that we printed here. You have the full posterior distributions for all parameters.  Therefore you also have a much more detailed understanding of how parameters relate to eachother (e.g., if two or more are correlated).

## Step 6: To do

### Go back to the start of Step 5 above. How well do you understand each step? What happens if you change different values? 

For example:
* Change the number of iterations
* Change the sampling rate (Nthin)
* Change nburn
* etc.

As you make these changes, see if you can continue to deepen your understanding of how emcee works and what the results mean. Don't hesitate to ask questions.

# Congratulations! You've completed Part 6!

## Extension Activity: Download observations and perform your own fit to a real-life binary star!

### Step 1: Download some data

One nice set of observed data is <a href="http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=J%2FAJ%2F137%2F3743&target=brief&">Table 1 from Geller et al. (2009)</a> on Vizier. 

Note: You may find it convenient to modify the table to only include SB1 binaries:<br>
cat Geller2009_table1.dat | awk '{if (NF == 7) print $0}' >! Geller2009_table1.dat.mod

Also, if you want to you ascii.read, you'll need to add a header

We've done all this for you, and placed <a href="data/Geller2009_table1.dat.mod">Geller2009_table1.dat.mod</a> in your data folder.

### Step 2: Create cells below (following what we did above) to fit an orbit to these data

Note: one more elegant way to code in python is to create a <a href="https://docs.python.org/2/tutorial/classes.html"> Python class</a>.  For instance, you could create a class that has this entire fitter in it.  And then you can use that fitter for any set of data.  I encourage you to try this!  If you are interested and have questions, please ask.

#### You can check how well your results match the fits from our "frequentist" method by looking up these binaries in <a href="http://adsabs.harvard.edu/abs/2009AJ....137.3743G">our paper</a>!


### Further Extension Activity: Find another set of observed data to run MCMC on :)